In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from numpy import linalg as la
import sys
import matplotlib.pyplot as plt
import seaborn as sas
import os
from scipy.ndimage import gaussian_filter
import csv


In [2]:
# dict Position, file
label_Exp1 = {
    '1': 'Supine',
    '2': 'Right',
    '3': 'Left',
    '4': 'Right',
    '5': 'Right',
    '6': 'Left',
    '7': 'Left',
    '8': 'Supine',
    '9': 'Supine',
    '10': 'Supine',
    '11': 'Supine',
    '12': 'Supine',
    '13': 'Right',
    '14': 'Left',
    '15': 'Supine',
    '16': 'Supine',
    '17': 'Supine'
}


In [3]:
class Dataset:
    def __init__(self, values, subject, position):
        self.values = values
        self.subject = subject
        self.position = position
        

In [4]:
def PCA(file):
    df = pd.read_csv(file, sep="\t", header=None)
    #la stessa posizione viene misurata 82 volte DC
    df.drop(columns=[2048], inplace = True) #tolgo colonna
    df.drop(0, inplace = True) #tolgo riga
    df.drop(1, inplace = True) #tolgo riga
    df.drop(2, inplace = True) #tolgo riga
    
    array = df.to_numpy()
    avg = np.mean(array, axis = 1, keepdims=True)

    array = array - avg

    CX = np.cov(array)
    w,v = la.eig(CX) 

    CY =  np.diag(np.round(np.real(w),2))

    P = v[:,0:1]
    P = abs(P)

    array_final = P.T.dot(array)
    
    array_final[0] = array_final[0]
    array_final[0][array_final[0] < 0] = 0
    array_final[0][array_final[0] > 3000] = 1000

    OldRange = array_final[0].max()  
    NewRange = 1000 
    for i in range(0,len(array_final[0])):
        array_final[0][i] = (((array_final[0][i]) * NewRange) / OldRange) 
        
    return array_final[0]

def gaussianFilter(PCA_result):
    gauss = gaussian_filter(PCA_result, sigma=1.5)
    
    return gauss
    
def padding(pressure_map):
    old_map = np.reshape(27,64)
    start = np.zeros(3, 64)
    final = np.zeros(2, 64)
    new_map = np.hstack(start, old_map, final)
    
    return new_map

def flip(pressure_map):
    pressure_map = np.flip(pressure_map, 0)
    
    return pressure_map
    
def trueLabel(num_file):
    position = label_Exp1.get(num_file)
    
    return position

def clean_string(s):
    file = s[24:27]
    
    if file[2] == '.':
        file = file[0:2]

    if file[1:2] == '.':
        file = file[0]

    if file[0] == '/':
        file = file[1]

    return file
            

In [5]:
rootdir = Path("dataset/experiment-i/")
pressure_map = []
dataset = []


for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print(os.path.join(subdir, file))
        file_name = os.path.join(subdir, file)
        
        pressure_map = PCA(file_name)
        pressure_map = gaussianFilter(pressure_map)
        
        subject = file_name[22:24]
        if subject[1:2] == '/':
            subject = subject[0]
        
        file = clean_string(file_name)
        position = trueLabel(file)
        
        data = Dataset(pressure_map, subject, position)
        dataset.append(data)
        

In [6]:
'''# collect data from experiment 2
rootdir = Path("dataset/experiment-ii/")
pressure_map = []
dataset = []


for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(os.path.join(subdir, file))
        file_name = os.path.join(subdir, file)
        
        pressur_map = flip()
        pressure_map = gaussianFilter(pressure_map)
        
        subject = file_name[22:24]
        if subject[1:2] == '/':
            subject = subject[0]
        
        file = clean_string(file_name)
        position = trueLabel(file)
        
        data = Dataset(pressure_map, subject, position)
        dataset.append(data)
        '''

'# collect data from experiment 2\nrootdir = Path("dataset/experiment-ii/")\npressure_map = []\ndataset = []\n\n\nfor subdir, dirs, files in os.walk(rootdir):\n    for file in files:\n        print(os.path.join(subdir, file))\n        file_name = os.path.join(subdir, file)\n        \n        pressur_map = flip()\n        pressure_map = gaussianFilter(pressure_map)\n        \n        subject = file_name[22:24]\n        if subject[1:2] == \'/\':\n            subject = subject[0]\n        \n        file = clean_string(file_name)\n        position = trueLabel(file)\n        \n        data = Dataset(pressure_map, subject, position)\n        dataset.append(data)\n        '

In [7]:
# write in csv
headers = []

for i in range(1, 65):
    for j in range(1, 33):
        header = "{} x {}".format(i, j)
        headers.append(header)

headers.append('Subject')
headers.append('Position')


with open('dataset.csv', mode='w') as csv_file:
    fieldnames = headers
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    for item in dataset:
        row = {}
        
        for i in range(0, len(item.values) + 2):
            
            if headers[i] == 'Subject':
                row[headers[i]] = item.subject

            elif headers[i] == 'Position':
                row[headers[i]] = item.subject

            else:
                row[headers[i]] = item.values[i]
        
        writer.writerow(row)
            